# Figure 5. Regional glacier influence (2000-2019)

In [1]:
import os 
import numpy as np
import pandas as pd
import geopandas as gpd

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import shapely.geometry

os.chdir('/home/rooda/OneDrive/Projects/DeepHydro/')
path_pmet = "/home/rooda/OneDrive/PatagoniaMet/"
path_geo = "/home/rooda/Datasets/GIS/"

## Data

In [2]:
pmet_shape = gpd.read_file("data/GIS/Basins_Patagonia_all_data.gpkg")
pmet_shape = pmet_shape.set_index("gauge_id")

# from km2 to m2
pmet_shape["total_area"] = pmet_shape.total_area * 1e6 

In [3]:
# glacier runoff (in m3/s)
data_runoff = pd.read_csv("results/runoff/glacier_runoff_historical_all.csv", index_col = 0, parse_dates= ["time"])
data_runoff = data_runoff[data_runoff.index.year >= 2000]
data_runoff = data_runoff.resample("YS").mean()
data_runoff = data_runoff.mean().rename("runoff_mean")

# glacier runoff (in m3/s)
data_melt = pd.read_csv("results/runoff//glacier_melt_historical_all.csv", index_col = 0, parse_dates= ["time"])
data_melt = data_melt[data_melt.index.year >= 2000]
data_melt = data_melt.resample("YS").mean()
data_melt = data_melt.mean().rename("melt_mean")

In [4]:
pmet_shape   = pd.concat([pmet_shape, data_runoff, data_melt], axis = 1) 
pmet_shape   = pmet_shape.fillna(0)

# from m3/s to mm/year
pmet_shape["runoff_mean"] = pmet_shape.runoff_mean*365*1000*86400 / pmet_shape.total_area 
pmet_shape["melt_mean"] = pmet_shape.melt_mean*365*1000*86400 / pmet_shape.total_area 

# glacier relative
pmet_shape["runoff_relative"] = pmet_shape.runoff_mean / pmet_shape.p_mean_PMET
pmet_shape["melt_relative"] = pmet_shape.melt_mean / pmet_shape.runoff_mean

## Plot elements

In [5]:
# basemap for background
geo_map = gpd.read_file(path_geo + "/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -54.99), (-76, -40.51), (-68.01, -40.51), (-68.01, -54.99), (-76, -54.99)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [6]:
pmet_shape_main = pmet_shape[pmet_shape["total_area"] > 5000*1e6]
pmet_shape_main["basin_name"] = ["Santa Cruz                          ", "Puelo", "Yelcho", "Grey","Palena", "Pascua", "Aysen", "Baker", "Cisnes"]

## Plot

In [7]:
# colors
cl = px.colors.qualitative.D3
cs = px.colors.colorbrewer.GnBu

fig = make_subplots(rows=2, cols=3, vertical_spacing = 0.05, shared_xaxes = True, horizontal_spacing = 0.01, 
                    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "scattergeo", "rowspan": 2}, {"type": "scattergeo", "rowspan": 2}], 
                                                      [None,                                 None, None]])

dtick = 2
x = list(range(-76, 0 + dtick, dtick))
y = list(range(-56, 0 + dtick, dtick))
xpos = -75.9
ypos = -56

## Basemap
for x_plot in range(0,3):
    fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                                colorscale = ["#EAEAF2", "#EAEAF2"],  showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=x_plot+1)

    fig.add_trace(go.Scattergeo(lon = [lon + 0.25 for lon in x[1:-1]] + [xpos + 0.25] * (len(y) - 2),
                                lat = [ypos + 0.1] * (len(x) - 2) + [lat + 0.1 for lat in y[1:-1]],
                                showlegend = False,
                                text = x[1:-1] + y[1:-1], textfont=dict(size=11, color = "rgba(0,0,0,0.25)"),
                                mode = "text"), row=1, col=x_plot+1)

fig.add_scattergeo(geojson = eval(pmet_shape_main['geometry'].to_json()), locations = pmet_shape_main.index, text = pmet_shape_main['basin_name'], mode = 'text', 
                       showlegend = False, textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=1)

colorbar_config_a = dict(title='Q<sub>g</sub><sup>runoff</sup><br>(mm yr<sup>-1</sup>)', len=0.45, x=0.23, y= 0.77, thickness=20, tickwidth = 1, 
                         ticktext = ["10<sup>0", "10<sup>1", "10<sup>2", "10<sup>3", "10<sup>4"], tickvals =  [0,1,2,3,4])

fig.add_trace(go.Choropleth(geojson = eval(pmet_shape['geometry'].to_json()),  locations = pmet_shape.index, z = np.log10(pmet_shape['runoff_mean']+0.001), 
                            colorscale = [cs[1], cs[4], cs[8]], marker_line_color ='white', marker_line_width=0.2, 
                            zmin = 0, zmax = 4, colorbar= colorbar_config_a), row=1, col=1)

colorbar_config_b = dict(len=0.45, x=0.55, y= 0.77, title='Ratio<br>(Q<sub>g</sub><sup>melt</sup>/ Q<sub>g</sub><sup>runoff</sup>)', thickness=20, tickwidth=1, dtick = 15, ticksuffix = "%")
fig.add_trace(go.Choropleth(geojson = eval(pmet_shape['geometry'].to_json()),  locations = pmet_shape.index, z = pmet_shape['melt_relative'].fillna(0)*100, 
                            colorscale = [cs[1], cs[4], cs[8]], marker_line_color ='white', marker_line_width=0.2, 
                            zmin = 20, zmax = 80, colorbar= colorbar_config_b), row=1, col=2)

colorbar_config_c = dict(len=0.45, x=0.89, y= 0.77, title='Ratio<br>(Q<sub>g</sub><sup>runoff</sup>/ PP)', thickness=20, tickwidth=1, ticktext = ["0%", "25%", "50%", "75%", ">100%"], tickvals =  [0,25,50,75,100])
fig.add_trace(go.Choropleth(geojson = eval(pmet_shape['geometry'].to_json()),  locations = pmet_shape.index, z = pmet_shape['runoff_relative']*100, 
                            colorscale = [cs[1], cs[4], cs[8]], marker_line_color ='white', marker_line_width=0.2, 
                            zmin = 0, zmax = 100, colorbar=colorbar_config_c), row=1, col=3)


# Layout -------------------------------------------------------------------------------------------------------------------
fig.update_xaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)
fig.update_yaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)

fig.update_geos(showframe = True, framewidth = 1, framecolor = "rgba(0,0,0,0.5)", 
                lonaxis_range=[-76, -68], lataxis_range=[-56, -40.5], 
                bgcolor = "rgb(255,255,255)", 
                showland = False, showcoastlines = False, showlakes = False, 
                lataxis_showgrid=True, lonaxis_showgrid=True, 
                lonaxis_dtick = dtick, lataxis_dtick = dtick, 
                lonaxis_gridcolor = "rgba(0,0,0,0.1)", lataxis_gridcolor = "rgba(0,0,0,0.1)", 
                lonaxis_griddash = "dot", lataxis_griddash = "dot")

fig.add_annotation(text="(a)", font=dict(size=16), x=0.005, y=0.99, xref = "paper", yref = "paper", showarrow=False)
fig.add_annotation(text="(b)", font=dict(size=16), x=0.35, y=0.99,  xref = "paper", yref = "paper", showarrow=False)
fig.add_annotation(text="(c)", font=dict(size=16), x=0.70, y=0.99,  xref = "paper", yref = "paper", showarrow=False)

fig.update_layout(autosize = False, width = 1000, height = 650, template = "seaborn", margin = dict(l=5, r=5, b=5, t=5))
fig.write_image("reports/figures/Figure5_Regional_glacier_influence.png", scale = 4)
#fig.show()

## Text

In [ ]:
data_runoff = pd.read_csv("results/runoff/glacier_runoff_historical_all.csv", index_col = 0, parse_dates= ["time"])
data_runoff = data_runoff[data_runoff.index.year >= 2000]
data_runoff = data_runoff.resample("YS").mean().sum(axis = 1)

"The  glacier runoff for the study area is {:.0f} +- {:.0f} mm yr-1".format(data_runoff.mean()*365*1000*86400/pmet_shape.total_area.sum(), 
                                                                                   data_runoff.std()*365*1000*86400/pmet_shape.total_area.sum())

In [ ]:
"greater than 1000 mm yr-1 in {:.1f}% of the study area".format(
    pmet_shape[pmet_shape['runoff_mean'] > 1000].total_area.sum()*100/pmet_shape.total_area.sum())

In [ ]:
data_runoff = pd.read_csv("results/runoff/glacier_runoff_historical_all.csv", index_col = 0, parse_dates= ["time"])
data_runoff = data_runoff[data_runoff.index.year >= 2000]

data_runoff_jdf = data_runoff[data_runoff.index.month.isin([1, 2, 12])]
data_runoff_jdf = data_runoff_jdf.resample("YS").mean().sum(axis = 1)
data_runoff = data_runoff.resample("YS").mean().sum(axis = 1)

"The total glacier runoff flux for the study area is {:.0f} +- {:.0f} m3/s (in summer {:.0f} +- {:.0f} m3/s)".format(
    data_runoff.mean(), data_runoff.std(),
    data_runoff_jdf.mean(), data_runoff_jdf.std())

# note high inter annual variablity


In [ ]:
data_runoff = pd.read_csv("results/runoff/glacier_melt_historical_all.csv", index_col = 0, parse_dates= ["time"])
data_runoff = data_runoff[data_runoff.index.year >= 2000]

data_runoff_jdf = data_runoff[data_runoff.index.month.isin([1, 2, 12])]
data_runoff_jdf = data_runoff_jdf.resample("YS").mean().sum(axis = 1)
data_runoff = data_runoff.resample("YS").mean().sum(axis = 1)

"The total glacier melt flux for the study area is {:.0f} +- {:.0f} m3/s (in summer {:.0f} +- {:.0f} m3/s)".format(
    data_runoff.mean(), data_runoff.std(),
    data_runoff_jdf.mean(), data_runoff_jdf.std())


In [ ]:
(((pmet_shape_main.runoff_mean * pmet_shape_main.total_area / (365*1000*86400))*100 / data_runoff.mean()).set_axis(pmet_shape_main["basin_name"])).round(1)

In [ ]:
"The mean ratio melt / total is {:.1f}% (weigthed glacier area)".format(
    (pmet_shape['melt_relative'] * pmet_shape.glacier_cover*0.01 * pmet_shape.total_area).sum()*100 /  (pmet_shape.glacier_cover*0.01 * pmet_shape.total_area).sum())

In [ ]:
"When compared to total climatic input, the ratio between glacier runoff and precipitation was greater than 50% and 10% in {:.1f}% and {:.1f}% of the total area, respectively".format(
    pmet_shape.total_area[pmet_shape['runoff_relative'] > 0.5].sum() *100 / pmet_shape.total_area.sum(),
    pmet_shape.total_area[pmet_shape['runoff_relative'] > 0.1].sum() *100 / pmet_shape.total_area.sum())